# Advanced techniques

Let's explore feature engineering techniques with the house prices dataset from Kaggle.

We can find an illustrative example of how to use Deep feature synthesis [here](https://www.kaggle.com/willkoehrsen/featuretools-for-good), and a good explanation [here](https://stackoverflow.com/questions/52418152/featuretools-can-it-be-applied-on-a-single-table-to-generate-features-even-when).

To Do
- Group some features together (business domain)
- Find categories under-represented (beyond current proposed method)
- ~~Scale numerical features~~
- Onehot/Label binarize encode categorical features (option)
- Baseline the model with a multiple linear regression and high degree polynomials

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Advanced-techniques" data-toc-modified-id="Advanced-techniques-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Advanced techniques</a></span><ul class="toc-item"><li><span><a href="#Setup-the-dataset" data-toc-modified-id="Setup-the-dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup the dataset</a></span><ul class="toc-item"><li><span><a href="#Feature-Creation" data-toc-modified-id="Feature-Creation-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Feature Creation</a></span></li><li><span><a href="#Scale-numerical-features" data-toc-modified-id="Scale-numerical-features-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Scale numerical features</a></span></li><li><span><a href="#Check-skewness" data-toc-modified-id="Check-skewness-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Check skewness</a></span></li><li><span><a href="#Check-correlation" data-toc-modified-id="Check-correlation-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Check correlation</a></span></li><li><span><a href="#Under-represented-features" data-toc-modified-id="Under-represented-features-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Under represented features</a></span></li><li><span><a href="#OneHot-encoding-for-categorical-variables." data-toc-modified-id="OneHot-encoding-for-categorical-variables.-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>OneHot encoding for categorical variables.</a></span></li><li><span><a href="#Baseline-basic-all-numeric-features" data-toc-modified-id="Baseline-basic-all-numeric-features-1.1.7"><span class="toc-item-num">1.1.7&nbsp;&nbsp;</span>Baseline basic all-numeric features</a></span></li></ul></li><li><span><a href="#Deep-Feature-Synthesis" data-toc-modified-id="Deep-Feature-Synthesis-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Deep Feature Synthesis</a></span><ul class="toc-item"><li><span><a href="#Build-the-EntitySet" data-toc-modified-id="Build-the-EntitySet-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Build the EntitySet</a></span></li><li><span><a href="#Normalize-the-entity" data-toc-modified-id="Normalize-the-entity-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Normalize the entity</a></span></li><li><span><a href="#Deep-feature-synthesis" data-toc-modified-id="Deep-feature-synthesis-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Deep feature synthesis</a></span></li></ul></li></ul></li></ul></div>

## Setup the dataset

In [1]:
import nbimporter
import numpy as np
import featuretools as ft
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sns
import warnings

from copy import copy

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit, \
        validation_curve, cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from src.dataset import Dataset

In [2]:
houses = Dataset('./data/houseprices_prepared.csv.gz')
houses.describe()


Available types: [dtype('int64') dtype('O') dtype('float64')]
80 Features
43 categorical features
37 numerical features
16 categorical features with NAs
0 numerical features with NAs
64 Complete features
--
Target: Not set


We will replace the NA's in the dataset with 'None' or 'Unknown' since they're not really NA's. For no good reason the person in charge of encoding the file decided to assign NA's to values where the feature does not apply, but instead of using a value for that special condition (like the string 'None') he/she decided to use the actual NA.

In [3]:
houses.replace_na(column='Electrical', value='Unknown')
houses.replace_na(column=houses.names('categorical_na'), value='None')
houses.set_target('SalePrice')
houses.describe()


Available types: [dtype('int64') dtype('O') dtype('float64')]
79 Features
43 categorical features
36 numerical features
0 categorical features with NAs
0 numerical features with NAs
79 Complete features
--
Target: SalePrice


### Feature Creation

This is the part where we decide to remove or add features based on our knowledge on the data and phenomena being represented. In this case, we're removing the `Id` field, and summing up some of the numerical fields counting the nr. of bathrooms, porchs and square feet.

In [4]:
def sum_SF():
    columns_to_add = ['1stFlrSF','2ndFlrSF','BsmtFinSF1','BsmtFinSF2']
    houses.features['House_SF'] = houses.\
        features[columns_to_add].fillna('').sum(axis=1)
    houses.drop_columns(columns_to_add)

def sum_Porch():
    columns_to_add = ['OpenPorchSF','3SsnPorch','EnclosedPorch',
                      'ScreenPorch','WoodDeckSF']
    houses.features['Porch_sf'] = houses.features[columns_to_add].sum(axis=1)
    houses.drop_columns(columns_to_add)

def sum_Baths():
    columns_to_add = ['FullBath', 'BsmtFullBath', 'HalfBath', 'BsmtHalfBath']
    houses.features['Total_Baths'] = (
        houses.features ['FullBath'] + 
        houses.features ['BsmtFullBath'] + 
        (0.5*houses.features['HalfBath']) + 
        (0.5*houses.features['BsmtHalfBath']))
    houses.drop_columns(columns_to_add)

houses.drop_columns('Id')
sum_SF()
sum_Porch()
sum_Baths()
houses.metainfo()
houses.describe()


Available types: [dtype('int64') dtype('O') dtype('float64')]
68 Features
43 categorical features
25 numerical features
0 categorical features with NAs
0 numerical features with NAs
68 Complete features
--
Target: SalePrice


### Scale numerical features

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.

In [5]:
houses.scale()
houses.describe()


Available types: [dtype('float64') dtype('O')]
68 Features
43 categorical features
25 numerical features
0 categorical features with NAs
0 numerical features with NAs
68 Complete features
--
Target: SalePrice


### Check skewness

In many modeling scenarios, normality of the features in a dataset is desirable. Power transforms are a family of parametric, monotonic transformations that aim to map data from any distribution to as close to a Gaussian distribution as possible in order to stabilize variance and minimize skewness.

In [6]:
houses.ensure_normality()
houses.describe()


Available types: [dtype('float64') dtype('O')]
68 Features
43 categorical features
25 numerical features
0 categorical features with NAs
0 numerical features with NAs
68 Complete features
--
Target: SalePrice


### Check correlation

In [7]:
numericals_to_drop, corr_num = houses.numerical_correlated(threshold=0.7)
print('There are {} correlated columns to remove.'.format(
    len(numericals_to_drop)))
print(numericals_to_drop)
# houses.plot_corr_matrix(corr_num)

There are 4 correlated columns to remove.
['TotRmsAbvGrd', 'GarageArea', 'House_SF', 'Total_Baths']


In [8]:
categoricals_to_drop, corr_categ = houses.categorical_correlated(threshold=0.7)
print('There are {} correlated columns to remove.'.format(
    len(categoricals_to_drop)))
print(categoricals_to_drop)
# houses.plot_corr_matrix(corr_categ)

There are 2 correlated columns to remove.
['Exterior2nd', 'GarageCond']


In [9]:
houses.drop_columns(categoricals_to_drop + numericals_to_drop)
houses.describe()


Available types: [dtype('float64') dtype('O')]
62 Features
41 categorical features
21 numerical features
0 categorical features with NAs
0 numerical features with NAs
62 Complete features
--
Target: SalePrice


### Under represented features

In [10]:
urf = houses.under_represented_features()
print('Features with unrepresented categories:\n', urf)
houses.drop_columns(urf)
print(end='')
houses.describe();

Features with unrepresented categories:
 ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'PoolQC']

Available types: [dtype('float64') dtype('O')]
57 Features
36 categorical features
21 numerical features
0 categorical features with NAs
0 numerical features with NAs
57 Complete features
--
Target: SalePrice


### OneHot encoding for categorical variables.

Convert categorical variable into dummy/indicator variables. I use pandas `get_dummies` for this task. Beware of not using this before measuring correlation, as it will destroy your measurements.

In [11]:
houses_prepared = copy(houses)
houses.onehot_encode()
houses.describe()


Available types: [dtype('float64') dtype('uint8')]
242 Features
0 categorical features
242 numerical features
0 categorical features with NAs
0 numerical features with NAs
242 Complete features
--
Target: SalePrice


### Baseline basic all-numeric features

Time to assess what can a simple and multiple linear regression can do.

In [ ]:
X, y = houses.split(test_size=0.2)

In [ ]:
model = LinearRegression()
cv = ShuffleSplit(n_splits=1000, test_size=0.2, random_state=666)
scores = cross_val_score(model, 
                         X.train, y.train, 
                         cv=cv, 
                         scoring='r2')
print('Obtained {} positive R2 scores'.format(len(scores[scores > 0.0])))
print('Best Validation R2: {:.2f}'.format(max(scores)))
print('Avg. Validation R2: {:.2f}'.format(np.mean(scores[scores > 0.0])))

sns.distplot(scores[scores > 0.0], hist = False, kde = True, 
             kde_kws = {'shade': True, 'linewidth': 3});

It seems that we can evaluate our regression problem using CV, but with a 1'st degree polynomial which is clearly too simple for this problem. The symptom of this is the extremely negative values of the R2 scores obtained.

To improve our solution, let's build a linear regression model with a higher degree polynomial. To do so, a scikit learn pipeline is used, where PolynomialFeatures is used before the linear regression, to try out different `degree` polynomials.

`PolynomialFeatures` generates a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. For example, if an input sample is two dimensional and of the form [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].

In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=123)
pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False), 
    LinearRegression(n_jobs=-1)).fit(X.train, y.train)
scores = cross_val_score(pipeline, X.train, y.train,
                         scoring="r2", cv=cv)

print('Obtained {} positive R2 scores'.format(len(scores[scores > 0.0])))
print('Best CV R2: {:.2f}'.format(max(scores)))
print('Avg. CV R2: {:.2f} +/- {:.02}'.format(
    np.mean(scores[scores > 0.0]),
    np.std(scores[scores > 0.0])))
print('R2 in hold-out dataset: {:.2f}'.format(
    pipeline.score(X.test, y.test)))

With the information obtained in the cross validation process, I know that I'm not overfitting, so my results seem to be OK. The $R^2$ obtained is decent. But I don't know which of my splits is producing the best possible result. So, at this point, I can rely on a single fit, or I can try to use the model trained with the split that produces the best generalization error.

To do so:

  1. I use `cross_validate` method instead of `cross_val_score`, and I also specify that I want the estimator trained with each split to be returned
  2. I score all the different estimators to see which one is producing the best generalization error over the hold-out dataset (`X.train` and `y.train`).
  3. Compare the results obtained with a single estimator over the entire dataset, with the results obtained over a single split (the one producing the best results in generalization).

In [ ]:
pipeline = Pipeline([
    ('polynomials', PolynomialFeatures(degree=2, include_bias=False)), 
    ('linear_regression', LinearRegression(n_jobs=-1))])
pipeline.fit(X.train, y.train)
training_score = pipeline.score(X.test, y.test)
print('R2 from entire-dataset estimator: {:.2f}'.format(training_score))

# Obtain scores and estimators from different splits and use the best one.
scores = cross_validate(pipeline, 
                        X.train, y.train,
                        scoring=['r2'], 
                        cv=5,
                        return_estimator=True)
split_scores = [scores['estimator'][i].score(X.test, y.test) 
                for i in range(len(scores))]
index_best = split_scores.index(max(split_scores))
print('Best estimator R2 score: {:.2f}'.format(split_scores[index_best]))

## Deep Feature Synthesis
### Build the EntitySet

In [12]:
es = ft.EntitySet()
es = es.entity_from_dataframe(entity_id='houses', 
                              dataframe=pd.concat([houses_prepared.features, 
                                                   houses_prepared.target], 
                                                  axis=1),
                              index = 'Id')
es

2019-01-28 22:05:35,102 featuretools.entityset - WARNING    index Id not found in dataframe, creating new integer column


Entityset: None
  Entities:
    houses [Rows: 1460, Columns: 59]
  Relationships:
    No relationships

### Normalize the entity

In [13]:
all_variables = houses_prepared.names('all') + [houses_prepared.target.name]
es.normalize_entity(base_entity_id='houses', 
                    new_entity_id='houses_norm',
                    index='Id')
es

Entityset: None
  Entities:
    houses [Rows: 1460, Columns: 59]
    houses_norm [Rows: 1460, Columns: 1]
  Relationships:
    houses.Id -> houses_norm.Id

### Deep feature synthesis

In [14]:
f_matrix, f_defs = ft.dfs(entityset=es,
                          target_entity='houses_norm', 
                          verbose=1,
                          n_jobs=-1)

Built 204 features
EntitySet scattered to workers in 3.383 seconds
Elapsed: 00:09 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks

Exception ignored in: <generator object add_client at 0x119b83fc0>
RuntimeError: generator ignored GeneratorExit
tornado.application - ERROR - Exception in Future <Future cancelled> after timeout
Traceback (most recent call last):
  File "/Users/renero/Code/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 970, in error_callback
    future.result()
concurrent.futures._base.CancelledError


Remove new variables that might be related to the target

In [15]:
drop_cols = []
for col in f_matrix:
    if col == houses_prepared.target.name:
        pass
    else:
        if houses_prepared.target.name in col:
            drop_cols.append(col)
            
print('Need to drop columns:', drop_cols)
f_matrix = f_matrix[[x for x in f_matrix if x not in drop_cols]]

Need to drop columns: ['SUM(houses.SalePrice)', 'STD(houses.SalePrice)', 'MAX(houses.SalePrice)', 'SKEW(houses.SalePrice)', 'MIN(houses.SalePrice)', 'MEAN(houses.SalePrice)']


In [16]:
# Create correlation matrix
corr_matrix = f_matrix.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] >= 0.99)]

print('Need to remove {} columns with >= 0.99 correlation.'.format(len(to_drop)))
f_matrix = f_matrix[[x for x in f_matrix if x not in to_drop]]

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:61640 remote=tcp://127.0.0.1:61609>


Need to remove 63 columns with >= 0.99 correlation.


In [24]:
fs_df = pd.concat(
    [f_matrix, houses_prepared.features, houses_prepared.target], 
    axis=1)
fs = Dataset.from_dataframe(fs_df)
fs.describe()


Available types: [dtype('float64') dtype('int64') dtype('O')]
193 Features
72 categorical features
121 numerical features
0 categorical features with NAs
42 numerical features with NAs
151 Complete features
--
Target: Not set


In [25]:
fs.replace_na(column=fs.names('numerical_na'), value=0)
fs.set_target(houses_prepared.target.name)
fs.onehot_encode()
fs.describe()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:61623 remote=tcp://127.0.0.1:61609>



Available types: [dtype('float64') dtype('int64') dtype('uint8')]
562 Features
0 categorical features
562 numerical features
0 categorical features with NAs
0 numerical features with NAs
562 Complete features
--
Target: SalePrice


In [ ]:
X, y = fs.split()

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=123)
pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False), 
    LinearRegression(n_jobs=-1)).fit(X.train, y.train)
scores = cross_val_score(pipeline, X.train, y.train,
                         scoring="r2", cv=cv)

print('Obtained {} positive R2 scores'.format(len(scores[scores > 0.0])))
print('Best CV R2: {:.2f}'.format(max(scores)))
print('Avg. CV R2: {:.2f} +/- {:.02}'.format(
    np.mean(scores[scores > 0.0]),
    np.std(scores[scores > 0.0])))
print('R2 in hold-out dataset: {:.2f}'.format(
    pipeline.score(X.test, y.test)))